In [4]:
pip install requests beautifulsoup4 lxml pandas tqdm


In [5]:
# idf_country_crawler.py
import re, time, sys
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm

BASE = "https://diabetesatlas.org"
LIST_URL = f"{BASE}/data-by-location/country/"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/122.0 Safari/537.36"
}

def get_country_links():
    """Lấy toàn bộ link country từ trang 'Data by country'."""
    r = requests.get(LIST_URL, headers=HEADERS, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "lxml")
    links = []
    for a in soup.select("a[href*='/data-by-location/country/']"):
        href = a.get("href") or ""
        # chuẩn hóa tuyệt đối + lọc chỉ country detail (loại link danh mục/lặp)
        if "/data-by-location/country/" in href:
            full = urljoin(BASE, href.split("#")[0])
            # bỏ đúng trang liệt kê (kết thúc bằng /country/) và giữ dạng /country/<slug>/
            if re.search(r"/data-by-location/country/[^/]+/?$", full):
                links.append(full)
    # khử trùng lặp, sort cho reproducibility
    links = sorted(set(links))
    return links

def parse_at_a_glance_table(country_url):
    """Tải trang country và parse bảng 'At a glance' thành DataFrame dài."""
    r = requests.get(country_url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    html = r.text

    soup = BeautifulSoup(html, "lxml")

    # 1) Tìm heading 'At a glance', sau đó tìm <table> kế tiếp
    atag = None
    for tag in soup.find_all(["h2","h3"]):
        if "at a glance" in (tag.get_text(strip=True) or "").lower():
            atag = tag
            break

    table = None
    if atag:
        table = atag.find_next("table")

    # 2) Nếu không tìm thấy theo heading, fallback: đọc tất cả table rồi chọn cái có cột năm 2000/2011/2024/2050
    tables = []
    if table:
        try:
            tables = pd.read_html(str(table))
        except Exception:
            tables = []
    if not tables:
        try:
            tables = pd.read_html(html)
        except Exception:
            tables = []

    if not tables:
        return None  # không có bảng

    # 3) Chọn bảng có “dấu hiệu năm”
    candidate = None
    for df in tables:
        text_cols = " ".join(map(str, df.columns))
        row0 = " ".join(map(str, df.iloc[0].tolist())) if len(df) else ""
        blob = f"{text_cols} {row0}".lower()
        if any(y in blob for y in ["2000","2011","2024","2050"]):
            candidate = df.copy()
            break
    if candidate is None:
        # lấy bảng đầu tiên làm fallback
        candidate = tables[0].copy()

    # 4) Làm sạch header/nội dung: 
    # Một số trang có header đa dòng → cố gắng set tên cột rõ ràng
    # Thực tế thường có cột đầu mô tả indicator + 4 cột năm 2000/2011/2024/2050
    df = candidate.copy()
    # nếu cột đầu tiên không có tên, đặt là 'indicator'
    if df.columns.dtype == "object":
        cols = [str(c) for c in df.columns]
    else:
        cols = [f"col_{i}" for i in range(df.shape[1])]
    df.columns = cols

    # Nếu dòng đầu chứa chính tên cột (common with read_html), xử lý:
    # (Ví dụ: hàng 0 có ["", "2000","2011","2024","2050"])
    possible_years = {"2000","2011","2024","2050"}
    if set(map(str, df.iloc[0].tolist())) & possible_years:
        # đặt row0 thành header nếu hợp lý
        new_cols = [str(x).strip() for x in df.iloc[0].tolist()]
        # đảm bảo có một cột mô tả bên trái
        if not any(x.lower().startswith("indicator") for x in new_cols):
            if new_cols[0] in possible_years:
                new_cols = ["Indicator"] + new_cols
        df = df[1:].copy()
        # tránh trùng tên cột
        for i,c in enumerate(new_cols):
            if not c or c == "nan":
                new_cols[i] = f"col_{i}"
        df.columns = new_cols

    # Chuẩn hóa tên cột năm
    rename_map = {}
    for c in df.columns:
        cc = str(c).strip()
        if cc.endswith(".1"): cc = cc[:-2]
        if re.fullmatch(r"(?:year\s*)?2000", cc, re.I): rename_map[c] = "2000"
        if re.fullmatch(r"(?:year\s*)?2011", cc, re.I): rename_map[c] = "2011"
        if re.fullmatch(r"(?:year\s*)?2024", cc, re.I): rename_map[c] = "2024"
        if re.fullmatch(r"(?:year\s*)?2050", cc, re.I): rename_map[c] = "2050"
    if rename_map:
        df = df.rename(columns=rename_map)

    # Đảm bảo có cột mô tả indicator/section bên trái
    if not any(col.lower() in ["indicator", "metrics", "description"] for col in df.columns):
        df = df.rename(columns={df.columns[0]: "indicator"})

    # Loại bỏ hàng rỗng/Loading/NaN
    df = df.replace({"—":"-", "–":"-", "Loading...":None}).dropna(how="all")
    for col in ["2000","2011","2024","2050"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().replace({"-": None, "nan": None, "None": None})
            # bỏ dấu phẩy/thanh phân cách nghìn → float
            df[col] = df[col].str.replace(",", "", regex=False)
            # đôi khi có ký hiệu % → giữ số
            df[col] = df[col].str.replace("%", "", regex=False)
            # giá trị kiểu "1,071.0" đã xử lý, còn "1 071,0" (locale) xử lý thêm:
            df[col] = df[col].str.replace(" ", "", regex=False)
            # chuyển số
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Tách section nếu indicator chứa dòng section (ví dụ 'Diabetes estimates (20-79 y)')
    # Kỹ thuật: ffill section khi gặp dòng bắt đầu bằng chữ và không có số ở các cột năm
    df["section"] = None
    current_section = None
    year_cols = [c for c in ["2000","2011","2024","2050"] if c in df.columns]
    processed_rows = []
    for _, row in df.iterrows():
        left = str(row["indicator"]).strip()
        # hàng tiêu đề section khi toàn bộ year_cols đều NaN và left không rỗng
        if (not left.lower().startswith(("people with", "age-standardised", "deaths", "total", "proportion", "live births", "population", "prevalence"))
            and all(pd.isna(row[c]) for c in year_cols)):
            current_section = left
            continue
        row = row.copy()
        row["section"] = current_section
        processed_rows.append(row)
    df2 = pd.DataFrame(processed_rows)

    # Melt → long
    long = df2.melt(id_vars=["indicator","section"], value_vars=year_cols,
                    var_name="year", value_name="value")
    long["country_url"] = country_url
    # country name từ <title> hoặc breadcrumb
    title = soup.title.get_text().split("|")[0].strip() if soup.title else country_url.split("/")[-2].replace("-", " ").title()
    long["country"] = title

    # dọn bản ghi trống
    long = long.dropna(subset=["value"]).reset_index(drop=True)
    return long

def main():
    countries = get_country_links()
    print(f"Found {len(countries)} country pages")
    all_rows = []
    for url in tqdm(countries, desc="Crawling countries"):
        try:
            df_long = parse_at_a_glance_table(url)
            if df_long is None or df_long.empty:
                print(f"[WARN] no table for {url}")
                continue
            all_rows.append(df_long)
            time.sleep(0.5)  # lịch sự với server
        except Exception as e:
            print(f"[ERR] {url} -> {e}", file=sys.stderr)

    if not all_rows:
        print("No data collected.")
        return
    out = pd.concat(all_rows, ignore_index=True)

    # Chuẩn hoá cột indicator (loại ký tự thừa)
    out["indicator"] = out["indicator"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
    out["section"] = out["section"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

    # Sắp xếp columns + sort
    out = out[["country","year","section","indicator","value","country_url"]]
    out = out.sort_values(["country","section","indicator","year"])

    out.to_csv("idf_country_at_a_glance.csv", index=False, encoding="utf-8")
    print("Saved -> idf_country_at_a_glance.csv", out.shape)

if __name__ == "__main__":
    main()


Found 215 country pages


Crawling countries:   0%|          | 0/215 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_11656\3421133582.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(table))
Crawling countries:   0%|          | 1/215 [00:00<03:33,  1.00it/s]

[WARN] no table for https://diabetesatlas.org/data-by-location/country/afghanistan/


C:\Users\Admin\AppData\Local\Temp\ipykernel_11656\3421133582.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(table))
Crawling countries:   1%|          | 2/215 [00:01<03:25,  1.04it/s]

[WARN] no table for https://diabetesatlas.org/data-by-location/country/albania/


...

[WARN] no table for https://diabetesatlas.org/data-by-location/country/yemen/


C:\Users\Admin\AppData\Local\Temp\ipykernel_11656\3421133582.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(table))
Crawling countries: 100%|█████████▉| 214/215 [04:12<00:01,  1.93s/it]

[WARN] no table for https://diabetesatlas.org/data-by-location/country/zambia/


C:\Users\Admin\AppData\Local\Temp\ipykernel_11656\3421133582.py:59: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(table))
Crawling countries: 100%|██████████| 215/215 [04:13<00:00,  1.18s/it]

[WARN] no table for https://diabetesatlas.org/data-by-location/country/zimbabwe/
No data collected.
